In [12]:
#!pip install feedparser
# !pip install newspaper3k
# !pip install konlpy
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter
from bs4 import BeautifulSoup

In [16]:
# [단계#1] 모든 RSS파일을 돌아다니며 기사의 제목/link를 추출함
urls = ['http://rss.etnews.com/02.xml',
        'http://rss.etnews.com/02021.xml',
        'http://rss.etnews.com/02024.xml',
        'http://rss.etnews.com/02027.xml']
# 아래 함수는 RSS목록의 list안에 존재하는 모든 기사의 title, link를 list로 구성
def crawl_rss(urls):
    array_rss = [] # 함수 시작하는 시점에 빈 리스트를 만듦, 여기에 모든 기사를 채울 것임
    titles_rss = set() # [중복기사제거] 기사제목들의 집합을 구성(집합은 중복을 불허하기 떄문)
    for url in urls: # 주어진 4개의 RSS파일을 하나씩 방문(총 4번 들것이고,02, 02021, 02024, 02027)
        print('[Crawl URL]', url) # 현재 어디 있는지 출력
        parse_rss = feedparser.parse(url) #현재 url을 파싱한 후에 결과를 parse_rss에 저장
        for p in parse_rss.entries: # parse_rss에 있는 모든 entries/기사를 검색
            if p.title not in titles_rss:
                array_rss.append({'title':p.title, 'link':p.link}) # 기사에서 제목/link 추출후 리스트에 추가
                titles_rss.add(p.title) # [중복기사제거] 집합에 현재 기사제목이 없을 때만 추가
            else:
                print('Duplicated Article:', p.title) # [중복기사제거] 중복된다고 판단된 기사 제목 출력
    return array_rss
list_articles = crawl_rss(urls) # 실제 crawl_rss 실행
print(list_articles)

[Crawl URL] http://rss.etnews.com/02.xml
Duplicated Article: 2024 금융권 공동채용 박람회
Duplicated Article: 2024 금융권 공동채용 박람회
Duplicated Article: 2024 금융권 공동채용 박람회
Duplicated Article: 2024 금융권 공동채용 박람회
Duplicated Article: 2024 금융권 공동채용 박람회
Duplicated Article: 2024 금융권 공동채용 박람회
Duplicated Article: 2024 금융권 공동채용 박람회
[Crawl URL] http://rss.etnews.com/02021.xml
Duplicated Article: 산은, 동남권 지역특화 벤처플랫폼서 IR 개최
Duplicated Article: [ET라씨로] 유한양행, 폐암약 '렉카자' 美 FDA 승인…주가 오름세
Duplicated Article: 코넥스협, 2024년도 코넥스·VC 파트너스데이 개최
Duplicated Article: 거래소, 코로나19·엠폭스 테마주 '투자유의' 발동
Duplicated Article: “사람과 소통하세요” 은행권 시니어 응대 '아날로그 귀환'
Duplicated Article: 신한금융, 저출산 극복 위한 대체인력지원사업에 100억원 출연
Duplicated Article: [ET라씨로]LG엔솔, '배터리 안전진단 SW' 사업 확대에 상승세
Duplicated Article: [ET라씨로]HDC현대산업개발, 빠른 원가율 개선 기대에 상승세
Duplicated Article: “안 잡히면 대출한도 더 죈다”···가계부채 추가대책 예고
Duplicated Article: [ET라씨로] 티디에스팜, 코스닥 상장 첫날 228%↑
Duplicated Article: 상법 전문가 만난 이복현 원장…“한국기업 지배구조 특수성, 밸류업 걸림돌”
Duplicated Article: 금융권 채용 큰 장 섰다···공동채용 박람회 열려
Duplicat

In [21]:
# [단계 #2] list에 존재하는 모든 링크를 돌아다니면서 본분 text를 긁어오기
 # 아래의 함수는 하나의 url을 입력받아서, 링크를 타고 들어가, 그 안에서 title과 text를 추출
 # 언어는 한글로 기본 지정
 
def crawl_article(url, language='ko'):
    print('[Crawl Article]', url) # 현재 title과 text를 추출할 url을 프린트해줌
    a = Article(url, language=language) # Article을 사용하여 그 URL을 입력하고, 언어옵션지정(한글어기본), 결과를 a애 저장
    a.download()
    a.parse()
    return a.title, preprocessing(a.text) # a에 해당하는 title과 본문(text)을 출력한다

def preprocessing(text):
    # text_article = BeautifulSoup(text, 'html5lib').get_text() #html 태그 제거
    text_article = BeautifulSoup(text, 'lxml').get_text() #html 태그 제거
    return text_article

for article in list_articles: # list에 있는 모든 기사를 하나씩 방문
    _, text = crawl_article(article['link']) # 그 기사의 link를 crawl_article 함수에 넣어 본문(text) 추출
    article['text'] = text # 추출된 본문을 'text'라는 속성으로 새로 만들어서 저장
print(list_articles[0]) # 첫번째 기사를 출력하여 title, link, text가 모두 있는지 확인

[Crawl Article] https://www.etnews.com/20240821000346
[Crawl Article] https://www.etnews.com/20240821000322
[Crawl Article] https://www.etnews.com/20240821000267
[Crawl Article] https://www.etnews.com/20240821000259
[Crawl Article] https://www.etnews.com/20240821000227
[Crawl Article] https://www.etnews.com/20240821000212
[Crawl Article] https://www.etnews.com/20240821000171
[Crawl Article] https://www.etnews.com/20240821000031
[Crawl Article] https://www.etnews.com/20240821000196
[Crawl Article] https://www.etnews.com/20240821000187
[Crawl Article] https://www.etnews.com/20240821000186
[Crawl Article] https://www.etnews.com/20240821000185
[Crawl Article] https://www.etnews.com/20240821000180
[Crawl Article] https://www.etnews.com/20240821000176
[Crawl Article] https://www.etnews.com/20240821000174
[Crawl Article] https://www.etnews.com/20240821000172
[Crawl Article] https://www.etnews.com/20240821000134
[Crawl Article] https://www.etnews.com/20240821000129
[Crawl Article] https://www.